In [1]:
'''
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)
'''

'\nfunction ClickConnect(){\nconsole.log("Working"); \ndocument.querySelector("colab-toolbar-button#connect").click() \n}\nsetInterval(ClickConnect,60000)\n'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!cat /proc/meminfo

MemTotal:       26751684 kB
MemFree:        23942168 kB
MemAvailable:   25746820 kB
Buffers:           91768 kB
Cached:          1949396 kB
SwapCached:            0 kB
Active:           756000 kB
Inactive:        1729700 kB
Active(anon):     423484 kB
Inactive(anon):      376 kB
Active(file):     332516 kB
Inactive(file):  1729324 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:              7448 kB
Writeback:             0 kB
AnonPages:        444708 kB
Mapped:           245076 kB
Shmem:               928 kB
Slab:             185604 kB
SReclaimable:     131624 kB
SUnreclaim:        53980 kB
KernelStack:        4704 kB
PageTables:         6168 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:    13375840 kB
Committed_AS:    3104052 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
Percpu:             1840 kB
AnonHugePages:   

In [4]:
%load_ext rpy2.ipython

/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


In [ ]:
%%R
if (!requireNamespace("BiocManager", quietly = TRUE))
install.packages("BiocManager")
BiocManager::install("GSAR")

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/BiocManager_1.30.10.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 40205 bytes (39 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[

In [ ]:
%%R
library(tidyverse)
library(MASS)
library(GSAR)
library(parallel)

In [ ]:
%%R

# Simplex sample generator
simplex_gen  <- function(n, samples){
    simplex_out  <- matrix(, nrow = samples, ncol = n)
    for(r in c(1:samples)){
        sample  <- runif(n-1,0,1)  %>%  
                    sort(decreasing=FALSE)
        sample  <- append(0.0, sample)  %>% 
                    append(1.0)
        sample  <- sample[-1] - sample
        sample  <- sample[1:n]

        simplex_out[r,]  <- sample
     }
    return(simplex_out)
}

# Genberate hypercube
hyper_gen  <- function(n, samples){
    simplex_out  <- matrix(, nrow = samples, ncol = n)
    for(r in c(1:samples)){
        sample  <- runif(n,-1,1) 
        sample  <- sample[1:n]
        simplex_out[r,]  <- sample
     }
    return(simplex_out)
}

# MST for simplex
mst <- function(file_, nsamp=10000, perm=50){
    # Read files with samples
    df_  <- read_csv(file_)  %>%  
    dplyr::select(starts_with('x_'))  %>% 
    sample_n(size=nsamp, replace=FALSE)
    # Get dim of the read file
    n_rows  <- dim(df_)[1]
    n_cols  <- dim(df_)[2]
        if (n_rows > (nsamp-1)){
        simplex_sample  <- suppressWarnings(simplex_gen(n_cols,n_rows))
        
        df_  <- df_  %>% 
                data.matrix()   %>% 
                unname()
        
        df_  <- rbind(df_, simplex_sample)  
        df_  <- df_ %>% 
                aperm(c(2,1)) 
        result <- WWtest(object=df_, group=c(rep(1, n_rows),rep(2, n_rows)),
                         nperm=perm, pvalue.only=FALSE)
        z_df  <- tibble(dim=n_cols, z=result$statistic, 
                        p_value=result$p.value,
                             nsamp=nsamp)
        gc()
        return(z_df)
        
    }
    
}


In [ ]:
%%R
path  <- '/content/drive/My Drive/mhar_paper_samples_2/'
set.seed(5)
simplex_gen(5,10)
files <- list.files(path)
files  <- paste(path, files, sep='')

# Extract dimenions
m <- regexpr("%[0-9]+.csv", files, perl=TRUE)
dimension <- regmatches(files, m)
m <- regexpr("[0-9]+", dimension, perl=TRUE)
dimension <- regmatches(dimension, m)
dimension

#Extract figure
m <- regexpr("[0-9]+_[a-z]+%", files, perl=TRUE)
figure <- regmatches(files, m)
m <- regexpr("[a-z]+", figure, perl=TRUE)
figure <- regmatches(figure, m)
figure

# Extract seed
m <- regexpr("/[0-9]+_", files, perl=TRUE)
sed <- regmatches(files, m)
m <- regexpr("[0-9]+", sed, perl=TRUE)
sed <- regmatches(sed, m)

df_simulations <- tibble(file=files, n=dimension, figure=figure, seed=sed)

In [ ]:
%%R

save_path <- '/content/drive/My Drive/mst_5000/mst_2.csv'

# Create random seeds
seeds_ <- sample(1:10000000, 1000, replace=T)
s<- 1


## PArse existing experiments
if(!file.exists(save_path)){
    existance<- tibble(figure=character(),dim=numeric(), z=numeric(), 
                          p_value=numeric(),
                              nsamp=numeric(), file=character(), perm=numeric(),
               seed=numeric()) 
    
}else{
    existance <- read.csv(save_path) %>% dplyr::select( -contains('X'))
}

z_df <- existance

for(f in files){
  # Set hyperparameters
  set.seed(seeds_[s])
  file_name <- f
  perm <- 50
  n_sample <-5000
  print(file_name)
  df_  <- read_csv(file_name)
  df_ <- df_[c(5000:10000),]
  # Get dim of the read file
  n_rows  <- dim(df_)[1]
  n_cols  <- dim(df_)[2]
  print(n_rows)
  print(n_cols)

  # Check if the experiment already runned
  check_df <- FALSE
  if(f %in% existance$f){
  check_df <- existance %>% filter(file==f)
  check_df <- (check_df$dim == n_cols) && (check_df$nsamp == n_sample)  && (check_df$perm == perm)
  }

  check_df <- !check_df

  if(check_df){
    if (grepl('hyper', f, fixed = TRUE)){
    real_sample  <- suppressWarnings(hyper_gen(n_cols,n_rows))
    typ <- 'Hypercube'
  }
  if (grepl('simplex', f, fixed = TRUE)){
    real_sample  <- suppressWarnings(simplex_gen(n_cols,n_rows))
    typ <- 'Simplex'
  }

  df_  <- df_  %>% data.matrix()   %>% unname()
          
  df_  <- rbind(df_, real_sample)  
  df_  <- df_ %>% aperm(c(2,1)) 
  dim(df_)
  result <- WWtest(object=df_, group=c(rep(1, n_rows),rep(2, n_rows)),
                          nperm=2, pvalue.only=FALSE)
  z_df  <- tibble(figure=typ,dim=n_cols, z=result$statistic, 
                          p_value=result$p.value,
                              nsamp=n_sample, file=f, perm=perm, 
                  seed=seeds_[s]) %>% rbind(z_df)
  z_df %>% write.csv(save_path)
  gc()  
  
  }
  #Increment seed index
  s <- s + 1 
}


